In [27]:
## 创建共享内存
##%overwritefile
##%file:../src/Rnamedpipeproc.py
##%noruncode
def write2stdin(self,queue):
    def read_all_from_queue(queue):
        res = b''
        # sz=n
        size = queue.qsize()
        # if n=<size:size=sz
        while size != 0:
            res += queue.get_nowait()
            size -= 1
        return res,size
    bcontents,size=read_all_from_queue(queue)
    self.stdin.write(bcontents)
def write2sh(self,name,content):
    ## 放入队列里等待发送到共享内存
    self._fifosent_queue.put(content)
def fifoproc(self,fifoname=None,stdout2fifo=False,fifo2stdin=False,content=None):
    if (fifo2stdin or stdout2fifo) and (fifoname!=None and len(fifoname)>0):
        # if self.flock==None:self.flock=CFileLock(fifoname)
        if stdout2fifo:
            # and self.foflag!=True:
            # self._write_to_stdout("stdout2fifo--->"+str(stdout2fifo)+"\n")
            self.foflag=True
            self.write2sh(fifoname,content)
        if fifo2stdin:
            # and self.foflag!=True:
            # self._write_to_stdout("fifo2stdin--->"+str(fifo2stdin)+"\n")
            self.foflag=True
            self.write2stdin(self._fiforead_queue,fifoname)


[MyPythonKernel155401] Info:file h:\Jupyter\Myjupyter-kernel\base\../src/Rnamedpipeproc.py created successfully


In [28]:
## 创建共享内存读取线程
##%overwritefile
##%file:../src/fifo_threadproc.py
##%noruncode
## 创建共享内存
def fifo_threadproc(self,fifoname=None,stdout2fifo=False,fifo2stdin=False):
    if (fifo2stdin or stdout2fifo) and (fifoname!=None and len(fifoname)>0):
        # if self.flock==None:self.flock=CFileLock(fifoname)
        if self.kobj.flock==None:self.kobj.flock=CFileLock(fifoname)
        if stdout2fifo:
            ## 待发送到共享内存的数据队列
            
            self._sendend=False
            self._fifosent_queue = Queue()
            ## 启动把数据从 _sent_queue 队列发送到共享内存的线程
            self.kobj._logln("启动内存通信通道:"+fifoname)
            self._fifosend_thread = Thread(target=RealTimeSubprocess._send_data, args=(self,self._fifosent_queue,fifoname,4096))
            self._fifosend_thread.daemon = True
            self._fifosend_thread.start()

        if fifo2stdin:
            ## 接收到共享内存的数据队列
            self._fiforead_queue = Queue()
            ## 启动从共享内存把数据读到_read_queue队列的线程
            self.kobj._logln("启动内存通信通道:"+fifoname)
            self._fiforead_thread = Thread(target=RealTimeSubprocess._read_data, args=(self,self._fiforead_queue,fifoname,4096,self.outencode))
            self._fiforead_thread.daemon = True
            self._fiforead_thread.start()
@staticmethod            
def _send_data(robj,queue,name,memsize=1024):

    def read_all_from_queue(queue,n):
        robj._sendend=False
        res = b''
        size = queue.qsize()
        sz=size
        if n<=size:
            sz=n
            size=sz
        while sz != 0:
            res += queue.get_nowait()
            sz -= 1
        return res,size
    while not robj._stop_send_data:
        content,size=read_all_from_queue(queue,memsize)
        if size<1:
            robj._sendend=True
            time.sleep(1/1000)
            continue
        bret=robj.kobj.sendmsg2sh(name,memsize,content,robj)
    ## robj.kobj._logln("发送数据到内存通道({0})的线程退出了。".format(name))
        ## robj._sendend=True
@staticmethod
def _read_data(robj,queue,name,memsize=1024,outencode='UTF-8'):
    ## 从共享内存读取数据直到读取线程结束标志为止
    while not robj._stop_read_data:
        ## 从共享内存读取数据并保存到队列
        bret=robj.kobj.readdatafromsh(queue,name,memsize=memsize,outencode='UTF-8')


[MyPythonKernel155401] Info:file h:\Jupyter\Myjupyter-kernel\base\../src/fifo_threadproc.py created successfully


In [29]:
##%overwritefile
##%file:../src/RTSstdout_contents.py
##%noruncode
def out_stdout_contents(self,stdout_contents,magics):
    if stdout_contents:
        if self.kobj.get_magicsSvalue(magics,"outputtype").startswith("image"):
            self._write_to_stdout(stdout_contents,magics)
            ## reset outputtype
            magics['_st']["outputtype"]="text/plain"
            return
        contents=''
        if self.outencode=='UTF-8':
            contents = stdout_contents.decode('UTF-8', errors='ignore')
        else:
            contents = stdout_contents.decode(self.outencode, errors='ignore')
        # if there is input request, make output and then
        # ask frontend for input
        start = contents.find(self.__class__.inputRequest)
        if(start >= 0):
            contents = contents.replace(self.__class__.inputRequest, '')
            if(len(contents) > 0):
                self._write_to_stdout(contents,magics)
            readLine = ""
            while(len(readLine) == 0):
                readLine = self._read_from_stdin()
            # need to add newline since it is not captured by frontend
            readLine += "\n"
            self.stdin.write(readLine.encode())
        else:
            if self.stdout2fifo:
                self.fifoproc(self.fifoname,self.stdout2fifo,self.fifo2stdin,stdout_contents);
            self._write_to_stdout(contents,magics)


[MyPythonKernel155401] Info:file h:\Jupyter\Myjupyter-kernel\base\../src/RTSstdout_contents.py created successfully


In [30]:
##%overwritefile
##%file:../src/RealTimeSubprocess.py
##%noruncode
class RealTimeSubprocess(subprocess.Popen):
    """
    A subprocess that allows to read its stdout and stderr in real time
    """

    inputRequest = "<inputRequest>"
    kobj=None
    def setkobj(self,k=None):
        self.kobj=k
    def __init__(self, cmd, write_to_stdout, write_to_stderr, read_from_stdin,
        cwd=None,shell=False,env=None,kobj=None,outencode='UTF-8',
        fifoname=None,stdout2fifo=False,fifo2stdin=False):
        """
        :param cmd: the command to execute
        :param write_to_stdout: a callable that will be called with chunks of data from stdout
        :param write_to_stderr: a callable that will be called with chunks of data from stderr
        """
        self.fifoname=fifoname
        self.stdout2fifo=stdout2fifo
        self.fifo2stdin=fifo2stdin
        self.flock=None
        self.outencode=outencode
        self.kobj=kobj
        self._write_to_stdout = write_to_stdout
        self._write_to_stderr = write_to_stderr
        self._read_from_stdin = read_from_stdin
        if env!=None and len(env)<1:env=None

        self.foflag=False
        super().__init__(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, stdin=subprocess.PIPE,
                            bufsize=0,cwd=cwd,shell=shell,env=env)
        ## 待发送到共享内存的数据队列
        self._stop_send_data=False
        ## 接收到共享内存的数据队列
        self._stop_read_data=False
        ## 根据需要开启共享内存通信
        self.fifo_threadproc(self.fifoname,self.stdout2fifo,self.fifo2stdin)

        self._stdout_queue = Queue()
        self._stdout_thread = Thread(target=RealTimeSubprocess._enqueue_output, args=(self.stdout, self._stdout_queue,self))
        self._stdout_thread.daemon = True
        self._stdout_thread.start()

        self._stderr_queue = Queue()
        self._stderr_thread = Thread(target=RealTimeSubprocess._enqueue_output, args=(self.stderr, self._stderr_queue,self))
        self._stderr_thread.daemon = True
        self._stderr_thread.start()
    
    @staticmethod
    def _enqueue_output(stream, queue,robj):
        for line in iter(lambda: stream.read(4096), b''):
            queue.put(line)
        stream.close()
    def write_contents(self,magics=None):
        def read_all_from_queue(queue):
            res = b''
            size = queue.qsize()
            while size != 0:
                res += queue.get_nowait()
                size -= 1
            return res
        ## 处理内存通信读取到的内容并重定向到stdin
        if self.fifo2stdin:
            sh_contents = read_all_from_queue(self._fiforead_queue)
            if sh_contents:
                self.stdin.write(sh_contents)
        ## 处理错误输出的内容
        stderr_contents = read_all_from_queue(self._stderr_queue)
        if stderr_contents:
            if self.kobj!=None:
                self.kobj._logln(stderr_contents.decode('UTF-8', errors='ignore'),3)
            else:
                self._write_to_stderr(stderr_contents.decode('UTF-8', errors='ignore'))
        ## 处理标准输出的内容
        stdout_contents = read_all_from_queue(self._stdout_queue)
        self.out_stdout_contents(stdout_contents,magics)
    ##%include:../src/fifo_threadproc.py
    ##%include:../src/Rnamedpipeproc.py
    ##%include:../src/RTSstdout_contents.py
    def wait_end(self,magics):
        while self.poll() is None:
            if self.kobj.get_magicsSvalue(magics,"outputtype").startswith("text"):
                self.write_contents(magics)
            pass
            continue
        self.write_contents(magics)
        # self.write_contents(magics)
        # wait for threads to finish, so output is always shown
        self._stdout_thread.join()
        self._stderr_thread.join()
        ## 
        self.wait_stdoutd()
        self._stop_send_data=True
        self._stop_read_data=True
        # self.write_contents(magics)
        # self.clean_namedpipe()
        if self.kobj==None:
            self._write_to_stdout("The process end:"+str(self.pid)+"\n",magics)
        else:
            self.kobj._logln("The process end:"+str(self.pid))
        ############################################
        return self.returncode
    def wait_stdoutd(self):
        ## 等待发送队列处理完毕
        time.sleep(1/1000)
        if self.stdout2fifo:
            cmdstart_time = time.time()
            while not (self._fifosent_queue.empty() and self._sendend):
                run_time = time.time() - cmdstart_time
                if run_time > 60: 
                    self.kobj._logln("超时退出")
                    break
                time.sleep(1/1000)
                continue

[MyPythonKernel155401] Info:file h:\Jupyter\Myjupyter-kernel\base\../src/RealTimeSubprocess.py created successfully
